In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages
library(forecast)
library(fpp3)
library(tibble)
library(plyr)
library(dplyr)
library(tidyr)
library(lubridate)
library(ggplot2)
library(tsibbledata)
library(feasts)
library(ggfortify)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sales <- read.csv(file="../input/sales-time-series-forecasting-tx-afcs2021/sales_train_validation_afcs2021.csv")
calendar <- read.csv(file="../input/eigendata/calendar_afcs2021.csv")
submission <-read.csv(file="../input/sales-time-series-forecasting-tx-afcs2021/sample_submission_afcs2021.csv")
test <- read.csv(file="../input/sales-time-series-forecasting-tx-afcs2021/sales_test_validation_afcs2021.csv")

In [ ]:
# train + test combined
full <- merge(sales, test, by.x = "id", by.y = "id")

## this is the part necessary for the for loop

In [ ]:
#change date type
calendar$date <- as.Date(calendar$date, format="%m/%d/%Y" )

In [ ]:
sales_transpose <- as.data.frame(t(as.matrix(sales)))
test_transpose <- as.data.frame(t(as.matrix(test)))

In [ ]:
#makes id a new column adds index, changes the header to the first row, removes first row and resets index
sales_transpose <- cbind(newColName = rownames(sales_transpose), sales_transpose)
rownames(sales_transpose) <- 1:nrow(sales_transpose)

names(sales_transpose) <- as.matrix(sales_transpose[1, ])
sales_transpose <- sales_transpose[-1,]
rownames(sales_transpose) <- 1:nrow(sales_transpose)

In [ ]:
#makes id a new column adds index, changes the header to the first row, removes first row and resets index
test_transpose <- cbind(newColName = rownames(test_transpose), test_transpose)
rownames(test_transpose) <- 1:nrow(test_transpose)

names(test_transpose) <- as.matrix(test_transpose[1, ])
test_transpose <- test_transpose[-1,]
rownames(test_transpose) <- 1:nrow(test_transpose)

In [ ]:
# merge calendar & sales
df <- merge(calendar, sales_transpose, by.x = "d", by.y = "id")

# merge calendar & test
test_df <- merge(calendar, test_transpose, by.x='d', by.y='id')

In [ ]:
#select only date & food items & change classes
train <- df %>% select(2, 13:835) %>% retype()
test <- test_df %>% select(2, 13:835) %>% retype()

In [ ]:
# combining the train and test
df_full <- rbind(train,test)
# making a time series
df_full <- df_full %>% as_tsibble(index = date)

In [ ]:
#forecast ~ loop
h=28
ns=ncol(train)-1

mylist <- list()

for( i in 2:ns){
    food <- unlist(train[,i])
    fit <- train %>% as_tsibble(index=date) %>% model(mean=MEAN(food)) %>%forecast(h=h)
    mylist[[i]] <- fit$.mean
    }

# converting list to df
data <- as.data.frame(matrix(unlist(mylist), nrow = h, ncol = ns))

In [ ]:

# add row names to df
names= c('F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16','F17','F18','F19','F20','F21','F22','F23','F24','F25','F26','F27','F28')
rownames(data) = names

In [ ]:

data_transpose<- as.data.frame(t(as.matrix(data)))

In [ ]:

#creating new ID column to add to both df inorder to merge
num_rows=nrow(data_transpose)
ID <- c(1:num_rows)
 
# binding ID column to the both df's !!! run only 1 time
data_transpose <- cbind(ID , data_transpose)
submission <-cbind(ID, submission)

In [ ]:

# merging submission & data en deleting excessive columns & renames the columns
name = append("id", names)
new_submission <-  merge(submission, data_transpose, by.x='ID', by.y='ID') %>% select(-1, -(3:30)) 
colnames(new_submission) <- name

# write to csv 
#write.csv(new_submission,"/kaggle/working/trial_2.csv")


## faster way

In [ ]:
extract_ts <- function(df){' function to load the data'
  
  min_date <- date("2011-01-29")
  
  df %>%
    select(id, starts_with("d_")) %>%  
    pivot_longer(starts_with("d_"), names_to = "dates", values_to = "sales") %>%
    mutate(dates = as.integer(str_remove(dates, "d_"))) %>% 
    mutate(dates = min_date + dates - 1) %>% 
    mutate(id = str_remove(id, "_validation"))
  
}

In [ ]:
train_2 <- extract_ts(sales)
test_2 <- extract_ts(test)

In [ ]:
# make tsibble an give as key the id
train_2 <-train_2 %>% as_tsibble(index=dates, key=id)
test_2 <- test_2 %>% as_tsibble(index=dates, key=id)
full_2 <- rbind(train_2,test_2)

In [ ]:
# BASELINE models
system.time(fit_2_fc <-train_2 %>% model(mean=MEAN(sales),
                             snaive=SNAIVE(sales), 
                             naive=NAIVE(sales),
                             drift =RW(sales ~ drift())
                            ) %>% forecast(h=28))

In [ ]:
# accuracy for baseline models
acc_score_baseline <- accuracy(fit_2_fc, full_2)

In [ ]:
# save accuracy to csv
#write.csv(acc_score_baseline,"/kaggle/working/acc_score_baseline.csv", row.names=F)

In [ ]:
colMeans(acc_score_baseline %>% filter(.model == 'mean') %>% select(RMSE, MAE, MASE)) %>% round(digits = 3)
colMeans(acc_score_baseline %>% filter(.model == 'snaive') %>% select(RMSE, MAE, MASE)) %>% round(digits = 3)
colMeans(acc_score_baseline %>% filter(.model == 'naive') %>% select(RMSE, MAE, MASE)) %>% round(digits = 3)
colMeans(acc_score_baseline %>% filter(.model == 'drift') %>% select(RMSE, MAE, MASE)) %>% round(digits = 3)

In [ ]:
# fitting ARIMA & ETS
system.time(fit_3 <- train_2 %>% model(arima=ARIMA(sales), ets=ETS(sales)))

In [ ]:
# forecast for the arima & ets models
fit_3_fc <- fit_3 %>% forecast(h=28)

In [ ]:
# Save model as R file
#saveRDS(fit_3_fc, file = "fit_fc_ar_ets.RDS")

In [ ]:
# accuracy scores for arima & ets models
acc_score_ar_ets <- accuracy(fit_3_fc, full_2)

In [ ]:
# acc_score_baseline
colMeans(acc_score_baseline %>% filter(.model == 'snaive') %>% select(RMSE, MAE, SMAPE)) %>% round(digits = 3)
# SNAIVE: RMSE 2.322 MAE 1.697 MASE 1.724

colMeans(acc_score_baseline %>% filter(.model == 'mean') %>% select(RMSE, MAE, MASE)) %>% round(digits = 3)
# MEAN: RMSE 2.113 MAE 1.659 MASE 1.54

colMeans(acc_score_ets_ar %>% filter(.model == 'arima') %>% select(RMSE, MAE, MASE)) %>% round(digits = 3)
# ARIMA: RMSE 1.826 MAE 1.399 MASE 1.496
colMeans(acc_score_ets_ar %>% filter(.model == 'ets') %>% select(RMSE, MAE, MASE)) %>% round(digits = 3)
#ETS: RMSE 1.893 MAE 1.468 MASE 1.516

## For submission to kaggle

In [ ]:
t1 <- readRDS("../input/baseline/fit_fc_baseline.RDS")
t2 <- readRDS("../input/advanced/fit_fc_ar_ets.RDS")

In [ ]:
#for kaggle forecast the next 28 days
#fit_trial <- fit_3 %>% forecast(h=56)
#fit_trial_ets <- fit_trial %>% filter(.model == 'ets')
#fit_trial_arima <- fit_trial %>% filter(.model == 'arima')
fit_trial_snaive<-t1 %>% filter(.model =='snaive')

ns=823

list_my <- list()

for( i in 1:ns){
    
    list_my[[i]] <- fit_trial_snaive$.mean
    }

# converting list to df
data <- as.data.frame(matrix(unlist(list_my), nrow = 56, ncol = ns))

In [ ]:
data <- round(data, digits=2)
data_transpose<- as.data.frame(t(as.matrix(data)))
data_transpose <- data_transpose %>% select(-c(1:28))
head(data_transpose)

In [ ]:
# add row names to df
names= c('F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16','F17','F18','F19','F20','F21','F22','F23','F24','F25','F26','F27','F28')
colnames(data_transpose) = names


#creating new ID column to add to both df inorder to merge
num_rows=nrow(data_transpose)
ID <- c(1:num_rows)
 
# binding ID column to the both df's !!! run only 1 time
data_transpose <- cbind(ID , data_transpose)
submission <-cbind(ID, submission)

# merging submission & data en deleting excessive columns & renames the columns
name = append("id", names)
new_submission <-  merge(submission, data_transpose, by.x='ID', by.y='ID') %>% select(-1, -(3:30)) 
colnames(new_submission) <- name

# write to csv 
write.csv(new_submission,"/kaggle/working/SNAIVE.csv", row.names=F)



In [ ]:
# use system.time() to time how long it takes to run something